In [2]:
import json

with open('strava_key.json') as data_file:
    data = json.load(data_file)


client_id = data['client_id']
client_secret = data['client_secret']
access_token =  data['access_token']
refresh_token =  data['refresh_token']

auth_link = "http://www.strava.com/oauth/authorize?client_id="+client_id+"&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=profile:read_all,activity:read_all"
print(auth_link)

http://www.strava.com/oauth/authorize?client_id=98429&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=profile:read_all,activity:read_all


In [3]:
access_token = "35d474a5e8ad16a2d078c2722c7c0619fcb1e6eb"

In [4]:
import requests
import json# Make Strava auth API call with your 
# client_code, client_secret and code
response = requests.post(
                    url = 'https://www.strava.com/oauth/token',
                    data = {
                            'client_id': client_id,
                            'client_secret': client_secret,
                            'code': '35d474a5e8ad16a2d078c2722c7c0619fcb1e6eb',
                            'grant_type': 'authorization_code'
                            }
                )#Save json response as a variable
strava_tokens = response.json()# Save tokens to file
with open('strava_tokens.json', 'w') as outfile:
    json.dump(strava_tokens, outfile)# Open JSON file and print the file contents 
# to check it's worked properly
with open('strava_tokens.json') as check:
  data = json.load(check)
print(data)

{'message': 'Bad Request', 'errors': [{'resource': 'AuthorizationCode', 'field': 'code', 'code': 'invalid'}]}


# Activities CSV from athlete

In [6]:
import pandas as pd
import requests
import json
import time## Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)## If access_token has expired then use the refresh_token to get the new access_token
if strava_tokens['expires_at'] < time.time():#Make Strava auth API call with current refresh token
    response = requests.post(
                        url = 'https://www.strava.com/oauth/token',
                        data = {
                                'client_id': client_id,
                                'client_secret': client_secret,
                                'code': 'ffde4b99492188f4fd1f5b2756ff85c9b00261c1', #insert token from strava auth
                                'grant_type': 'authorization_code'
                                }
                    )#Save response as json in new variable
    new_strava_tokens = response.json()# Save new tokens to file
    with open('strava_tokens.json', 'w') as outfile:
        json.dump(new_strava_tokens, outfile)#Use new Strava tokens from now
    strava_tokens = new_strava_tokens#Loop through all activities
page = 1
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']## Create the dataframe ready for the API call to store your activity data
activities = pd.DataFrame(
    columns = [
            "id",
            "name",
            "start_date_local",
            "type",
            "distance",
            "moving_time",
            "elapsed_time",
            "total_elevation_gain",
            "end_latlng",
            "external_id"
    ]
)

while True:
    
    # get page of activities from Strava
    r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
    r = r.json()# if no results then exit loop
    if (not r):
        break
    
    # otherwise add new data to dataframe
    for x in range(len(r)):
        activities.loc[x + (page-1)*200,'id'] = r[x]['id']
        activities.loc[x + (page-1)*200,'name'] = r[x]['name']
        activities.loc[x + (page-1)*200,'start_date_local'] = r[x]['start_date_local']
        activities.loc[x + (page-1)*200,'type'] = r[x]['type']
        activities.loc[x + (page-1)*200,'distance'] = r[x]['distance']
        activities.loc[x + (page-1)*200,'moving_time'] = r[x]['moving_time']
        activities.loc[x + (page-1)*200,'elapsed_time'] = r[x]['elapsed_time']
        activities.loc[x + (page-1)*200,'total_elevation_gain'] = r[x]['total_elevation_gain']
        activities.loc[x + (page-1)*200,'end_latlng'] = r[x]['end_latlng']
        activities.loc[x + (page-1)*200,'external_id'] = r[x]['external_id']# increment page
    page += 1
    
activities.to_csv("strava_activities_"+client_id+".csv")

# getting gpx data from strava

In [7]:
import requests as r
import pandas as pd
import json
import gpxpy.gpx
from datetime import datetime, timedelta

#This info is in activity summary downloaded from API
id = 6403706752 #activity ID
start_time = '2022-01-01T12:04:10Z' #activity start_time_local

# Make API call
url = f"https://www.strava.com/api/v3/activities/{id}/streams"
header = {'Authorization': 'Bearer ' + access_token}
latlong = r.get(url, headers=header, params={'keys':['latlng']}).json()[0]['data']
time_list = r.get(url, headers=header, params={'keys':['time']}).json()[1]['data']
altitude = r.get(url, headers=header, params={'keys':['altitude']}).json()[1]['data']

# Create dataframe to store data 'neatly'
data = pd.DataFrame([*latlong], columns=['lat','long'])
data['altitude'] = altitude
start = datetime.strptime(start_time, "%Y-%m-%dT%H:%M:%SZ")
data['time'] = [(start+timedelta(seconds=t)) for t in time_list]

gpx = gpxpy.gpx.GPX()
# Create first track in our GPX:
gpx_track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(gpx_track)
# Create first segment in our GPX track:
gpx_segment = gpxpy.gpx.GPXTrackSegment()
gpx_track.segments.append(gpx_segment)
# Create points:
for idx in data.index:
    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(
                data.loc[idx, 'lat'],
                data.loc[idx, 'long'],
                elevation=data.loc[idx, 'altitude'],
                time=data.loc[idx, 'time']
    ))
# Write data to gpx file
with open(str(id)+'_.gpx', 'w') as f:
    f.write(gpx.to_xml())

In [23]:
import requests as r
import pandas as pd
import json
import gpxpy.gpx
from datetime import datetime, timedelta

def create_gpx(id, start_time, access_token):
    # Make API call
    url = f"https://www.strava.com/api/v3/activities/{id}/streams"
    header = {'Authorization': 'Bearer ' + access_token}
    latlong = r.get(url, headers=header, params={'keys':['latlng']}).json()[0]['data']
    time_list = r.get(url, headers=header, params={'keys':['time']}).json()[1]['data']
    altitude = r.get(url, headers=header, params={'keys':['altitude']}).json()[1]['data']

    # Create dataframe to store data 'neatly'
    data = pd.DataFrame([*latlong], columns=['lat','long'])
    data['altitude'] = altitude
    start = datetime.strptime(start_time, "%Y-%m-%dT%H:%M:%SZ")
    data['time'] = [(start+timedelta(seconds=t)) for t in time_list]

    gpx = gpxpy.gpx.GPX()
    # Create first track in our GPX:
    gpx_track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(gpx_track)
    # Create first segment in our GPX track:
    gpx_segment = gpxpy.gpx.GPXTrackSegment()
    gpx_track.segments.append(gpx_segment)
    # Create points:
    for idx in data.index:
        gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(
                    data.loc[idx, 'lat'],
                    data.loc[idx, 'long'],
                    elevation=data.loc[idx, 'altitude'],
                    time=data.loc[idx, 'time']
        ))
    # Write data to gpx file
    with open("db_rides/ride_"+str(start_time)+"_"+str(id)+'.gpx', 'w') as f:
        f.write(gpx.to_xml())

In [24]:
#access_token = "30d9f3bbcc4f1d74acb89c83ececa0a7fc0dd3a2"
i = 0
id = rides_2022[i]["id"]
start_time= rides_2022[i]["start_date_local"]

create_gpx(id, start_time,access_token)

In [11]:
from csv import DictReader
# open file in read mode
with open("strava_activities.csv", 'r') as f:
	dict_reader = DictReader(f)
	strava_dict = list(dict_reader)

	#print(list_of_dict)

# gather all real world rides from 2022 in one list
rides_2022 = []

for i in range(0,len(strava_dict)):
    if str(strava_dict[i]['type'])=='Ride' and str(strava_dict[i]['start_date_local']).split("-")[0]=="2022":
        rides_2022.append(strava_dict[i])

#example
rides_2022[0]["id"]


In [28]:
import time

for i in range(0,len(rides_2022)):
    id = rides_2022[i]["id"]
    start_time= rides_2022[i]["start_date_local"]

    create_gpx(id, start_time,access_token)
    print(str(ride["start_date_local"]))
    time.sleep(80)


SyntaxError: incomplete input (838052677.py, line 12)